<a href="https://colab.research.google.com/github/m10k1/gemini_cookbook/blob/main/Function_call%E3%81%AE%E5%9F%BA%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ｇｅｍｉｎｉ ＡＰＩ: ＰｙｔｈｏｎによるＦｕｎｃｔｉｏｎ コール

Ｆｕｎｃｔｉｏｎコールとは、開発者がＬＬＭに向けてコード内にある関数の説明文を作るようなものです。

ＬＬＭからの応答に説明文に該当する関数名とその引数が含まれて返ってきます。

Ｆｕｎｃｔｉｏｎコールは生成ＡＩアプリケーションで関数を使えるようにするための仕組みで、一つのリクエストで複数の関数を定義できます。

このノートブックではＦｕｎｃｔｉｏｎコールの基本的な使い方を学びます。

## セットアップ

pipモジュールを追加します

In [2]:
! pip install -qU google-genai

## API Keyの設定

ＧｅｍｉｎｉのＡＰＩを利用するにはＡＰＩキーを使用する必要があります。

In [7]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# クライアントオブジェクトの作成

from google import genai

client = genai.Client(api_key=GOOGLE_API_KEY)




## モデルの選択

ＧｅｍｉｎｉモデルはＦｕｎｃｔｉｏｎコールが利用できます。

In [9]:
MODEL_ID = "gemini-2.5-flash" #@param ["gemini-2.5-flash-lite", "gemini-2.5-flash-lite-preview-09-2025", "gemini-2.5-flash", "gemini-2.5-flash-preview-09-2025", "gemini-2.5-pro"] {"allow-input":true, isTemplate: true}


## ツールとして関数を設定する

Ｆｕｎｃｔｉｏｎコールを使うには、GenerativeModelを作成する際のtoolsパラメータに関数のリストを渡します。

モデルは、関数名、docstring, パラメータ、パラメータの型アノテーションを使ってプロンプトの解答するのに関数が必要かどうかを判断します。

> **重要**: ＳＤＫは関数パラメーターの型アノテーションをＡＰＩの様式に変換します（genai.types.FunctionDeclaration)。
> Python SDKの自動変換は次の型をサポートする。
AllowedTypes = int | float | bool | str | list['AllowdTypes']

**例: 照明システムの関数**

仮想照明システムを制御する3つの関数です。docstringと型のヒントに注意してください

In [8]:
def enable_lights():
  """照明を点灯します"""
  print("LIGHTBOT: Lights enabled.")

def set_light_color(rgb_hex: str):
  """照明の色を設定する。照明が点灯していなければならない"""
  print(f"LIGHTBOT: Lights set to {rgb_hex}")

def stop_lights():
  """照明を消灯します"""
  print("LIGHTBOT: 照明を消しました")

light_controls = [enable_lights, set_light_color, stop_lights]

instruction = """
あなたは便利な照明システムです。
照明を点灯したり消灯したりすることができます。そして照明の色も設定することができます。
それ以外の振る舞いは行わないこと
"""



## チャットでのＦｕｎｃｔｉｏｎコールの基本

バックグラウンドで何が起こったかを理解するには、チャット履歴を調べることができます。

Chat.historyプロパティには、ユーザとGeminiモデルの会話の時系列記録が保存されます。Chat.get_history()を使って履歴を取得できます。会話の各ターンは、以下の情報を含むgenai.types.Contentオブジェクトで表されます：

Role: コンテンツの発信元が "user "か "model "かを識別します。

パーツ： メッセージの個々のコンポーネントを表すgenai.types.Partオブジェクトのリスト。テキストのみのモデルでは、これらのパーツは以下のようになります：

テキスト： テキストメッセージ。
関数呼び出し（genai.types.FunctionCall）： 与えられた引数で特定の関数を実行するモデルからのリクエスト。
関数レスポンス(genai.types.FunctionResponse): 関数実行後にユーザが返す結果： リクエストされた関数を実行した後にユーザが返す結果。

In [14]:
chat = client.chats.create(
    model=MODEL_ID,
    config = {
        "tools": light_controls,
        "system_instruction": instruction,
    }
)

response = chat.send_message("ここの部屋は暗すぎる")

print(response.text)


LIGHTBOT: Lights enabled.
部屋の照明を点灯しました。


## Ｆｕｎｃｔｉｏｎコールの実行と履歴の調査

背後で何が起こっているのかを理解するために、チャットの履歴を調査することができます。

Chat.historyプロパティは時間軸順でユーザーとモデルの会話のレコードが保存されています。Chat.get_history()を使って履歴を取得できます。
各会話のターンはgenai.types.Contentオブジェクトで表現されます。
Contentオブジェクトには次の情報が含まれます。

* Role: コンテンツがuserからのものか、modelからのものか識別
* Parts: メッセージの個々のコンポーネントのgenai.types.Partオブジェクト
  - **Text**: プレーンテキスト
  - **Function Call(genai.typesFunctionCall)**: モデルからの関数実行の要求。
  - **Function Response(genai.types.FunctionResponse)**: 要求された関数の実行後にユーザーから戻す結果
  

In [18]:
from IPython.display import Markdown, display

def print_history(chat):
  print('print_history')
  for content in chat.get_history():
    display(Markdown("###" + content.role + ":"))
    for part in content.parts:
      if part.text:
        display(Markdown(part.text))
      if part.function_call:
        print(f"Functional call: {part.function_call}")
      if part.function_response:
        print(f"Function response: {part.function_response}")
    print("-" * 80)

print_history(chat)

print_history


###user:

ここの部屋は暗すぎる

--------------------------------------------------------------------------------


###model:

...

###user:

Function response: will_continue=None scheduling=None parts=None id=None name='enable_lights' response={'result': None}
--------------------------------------------------------------------------------


###model:

部屋の照明を点灯しました。

--------------------------------------------------------------------------------


ユーザーが入力した「ここの部屋は暗すぎる」というメッセージに対して
モデルは、Ｆｕｎｃｔｉｏｎコールを応答で返してきています。
Ｆｕｎｃｔｉｏｎコールで指定された関数名は、「enable_lights」で、ライトを点ける判断を行っていることが分かります。

ユーザー側ではＳＤＫによりChatSessionが自動的に実行されてenable_lights()が実行されます。automatic_function_callingが有効になっているため。

モデルは結果として、"部屋の照明を点灯しました。"を返します。

## 自動ファンクション実行（Python SDKの機能)

上記で示したように、Python SDKのChatSessionには自動関数実行という強力な機能があります。有効にすると（デフォルトです）、モデルがFunctionCallで応答した場合、SDKは次のことを行います：

1. 提供されているツールから対応するPython関数を検索します。

2. モデルによって提供された引数で関数を実行します。

3. 関数の戻り値をFunctionResponseでモデルに返します。

4. モデルの最終レスポンス（通常はテキスト）のみをコードに返します。

これにより、一般的なユースケースのワークフローが大幅に簡素化されます。

**例：数学オペレーション**


In [19]:
from google.genai import types

def add(a: float, b: float):
  """a + b の計算結果を返します"""
  return a + b

def subtract(a: float, b:float):
  """a - b の計算結果を返します"""
  return a - b

def multiply(a: float, b: float):
  """a * b  の計算結果を返します"""
  return a * b

def divide(a: float, b:float):
  """a / b の計算結果を返します"""
  return a / b

operation_tools = [add, subtract, multiply, divide]

chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": operation_tools,
        "automatic_function_calling": {"disable": False}
    }
)

response = chat.send_message(
    "私は57匹の猫を飼っていて、それぞれ44個のミトンを持っています。全部でいくつのミトンがありますか"
)

print(response.text)


2508個のミトンがあります。


In [20]:
print_history(chat)

print_history


###user:

私は57匹の猫を飼っていて、それぞれ44個のミトンを持っています。全部でいくつのミトンがありますか

--------------------------------------------------------------------------------


###model:

...

###user:

Function response: will_continue=None scheduling=None parts=None id=None name='multiply' response={'result': 2508}
--------------------------------------------------------------------------------


###model:

2508個のミトンがあります。

--------------------------------------------------------------------------------


## 自動関数スキーマ宣言

Python SDKの主な利便性は、Python関数から必要なFunctionDeclarationスキーマを自動的に生成する機能です。これは

* 関数名： (func.__name__)

* Docstring： 関数の説明に使用されます。

* パラメータ： 名前と型のアノテーション（int、str、float、bool、list、dict）。Google styleのような特定の書式を使用している場合）パラメータのDocstringは、説明を強化することもできます。

* 戻り値の型のアノテーション： 厳密にどの関数を呼び出すかを決定するためにモデルが使うわけではありませんが、良い習慣です。

一般的に、Python関数をツールとして直接使用する場合、FunctionDeclarationオブジェクトを手動で作成する必要はありません。

しかし、スキーマを検査したり、修正したり、Python関数オブジェクトをすぐに利用できないシナリオで使用する必要がある場合は、genai.types.FunctionDeclaration.from_callableを使用して明示的にスキーマを生成することができます。




In [21]:
import json

set_color_declaration = types.FunctionDeclaration.from_callable(
    callable = set_light_color,
    client = client
)

print(json.dumps(set_color_declaration.to_json_dict(), indent=4))

{
    "description": "\u7167\u660e\u306e\u8272\u3092\u8a2d\u5b9a\u3059\u308b\u3002\u7167\u660e\u304c\u70b9\u706f\u3057\u3066\u3044\u306a\u3051\u308c\u3070\u306a\u3089\u306a\u3044",
    "name": "set_light_color",
    "parameters": {
        "properties": {
            "rgb_hex": {
                "type": "STRING"
            }
        },
        "required": [
            "rgb_hex"
        ],
        "type": "OBJECT"
    }
}


## マニュアルＦｕｎｃｔｉｏｎコール

よりコントロールするため、あるいは自動関数呼び出しが利用できない場合、モデルからのgenai.types.FunctionCallリクエストを自分で処理することができます。以下のような場合です：

* デフォルトの "automatic_function_calling "でChatを使用している場合： {disable"： True}を使用している。
* Client.model.generate_contentを使用している（そしてチャット履歴を自分で管理している）。

例 映画

以下の例は、Pythonでシングルターンのcurlサンプルを呼び出す関数に大まかに相当します。おそらく仮想的なAPIから、映画の再生時間情報を（モックとして）返す関数を使用しています：


In [22]:
def find_movies(description: str, location: str):
    """任意の説明、ジャンル、タイトルの単語などに基づいて、現在映画館で上映されている映画のタイトルを検索します。

    Args:
        description: カテゴリーやジャンル、タイトル語、属性など、あらゆる種類の説明文
        location: 都市と州（例：カリフォルニア州サンフランシスコ、郵便番号：95616など
    """
    return ["Barbie", "Oppenheimer"]


def find_theaters(location: str, movie: str):
    """場所と、現在上映中の映画のタイトルから映画館を検索。
    Find theaters based on location and optionally movie title which are currently playing in theaters.

    Args:
        location: 都市と州（例：カリフォルニア州サンフランシスコ、郵便番号：95616など
        movie: 映画のタイトル
    """
    return ["Googleplex 16", "Android Theatre"]


def get_showtimes(location: str, movie: str, theater: str, date: str):
    """
    特定の劇場で上映される映画の開始時間を検索します。

    Args:
      location: 都市と州（例：カリフォルニア州サンフランシスコ、郵便番号：95616など
      movie: 映画のタイトル
      thearer: 劇場名
      date: 開演希望日
    """
    return ["10:00", "11:00"]

theater_functions = [find_movies, find_theaters, get_showtimes]

In [24]:

response = client.models.generate_content(
    model=MODEL_ID,
    contents="カリフォルニア州マウンテンビューでバービーの映画を上映している映画館は？",
    config = {
        "tools": theater_functions,
        "automatic_function_calling": {"disable": True}
    }
)

print(json.dumps(response.candidates[0].content.parts[0].to_json_dict(), indent=4))

{
    "thought_signature": "CqgDAdHtim_0o3DeEShO-hIN-toVw26NChscqHS7LRARchGMjJbQoR1Vzg3kgwMqaK4NvumvMr2yf-8K9a784gPeDFiX65DUJJ90sp9JSBrRYGEwm0hrTLrbo-zCX5m-i23JsHASEXT1PLCae-0es76IlHuW9m-5GyT51w8m3O3JgIe625LLpSnrgJQ3J-dcJUCIftT0D1K_FabUWrabkg_mRcshTOKn7CIeoELAyoWQ6UME4iD2AJLebV182Rj56cgdIS-XxKwI1Z852_HYsWdBjvmsYompjEZ5_CSB3FQNZB1BU7nJGbN-fb0GJCnYe4R8eMxv5fgBp-AYhrj4SDAePoKC5vxDLYIqX0QwcWJhXEoqqslarft2jupFX6YBdzbxGTZv0MOp5tczpYROqs1s0GdVh5lyRanOT9rPMMD9wbvQ_3OZPojHRQGiL1bYPuR_VsqgksP5XELMd1WuNGSDvt_FRgR_FWA9yVqQ_heMbJWTLXLZIMpNWQSgqBXFaSTmGDjufOKdltGFvnpieoXzc79dgID4_LdsqKvlBwDt6VciDvfvG3117w==",
    "function_call": {
        "args": {
            "location": "Mountain View, California",
            "movie": "Barbie"
        },
        "name": "find_theaters"
    }
}


これは自動関数呼び出しの ChatSession を使用していないので、自分で関数を呼び出す必要があります。

これを行う非常に簡単な方法は if 文です：

if function_call.name == 'find_theaters'：
  find_theaters(**function_call.args)
elif ...
しかし、あなたはすでにtheater_functionsリストを作っているので、これは次のように単純化できる：

In [25]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    # Find the function object from the list based on the function name
    for func in functions:
        if func.__name__ == function_name:
            return func(**function_args)

part = response.candidates[0].content.parts[0]

# Functionコールかどうか確認。 実際の処理ではモデルの応答内容に応じてテキストを処理する必要がある
# .
if part.function_call:
    result = call_function(part.function_call, theater_functions)

print(result)

['Googleplex 16', 'Android Theatre']


最後に、モデルから最終的なテキストレスポンスを取得するために、レスポンスとメッセージ履歴を次の generate_content() 呼び出しに渡します。次のコードセルでは、Contentを書き出すさまざまな方法を示しています。

In [26]:
from google.genai import types
# Build the message history
messages = [
    types.Content(
        role="user",
        parts=[
            types.Part(
                text="Which theaters in Mountain View show the Barbie movie?."
            )
        ]
    ),
    types.Content(
        role="model",
        parts=[part]
    ),
    types.Content(
        role="tool",
        parts=[
            types.Part.from_function_response(
                name=part.function_call.name,
                response={"output":result},
            )
        ]
    )
]

# Generate the next response
response = client.models.generate_content(
    model=MODEL_ID,
    contents=messages,
    config = {
        "tools": theater_functions,
        "automatic_function_calling": {"disable": True}
    }
)
print(response.text)

「バービー」を上映しているマウンテンビューの映画館は、Googleplex 16とAndroid Theatreです。


## 並列なファンクションコール

Gemini APIは、1つのターンで複数の関数を呼び出すことができる。これは、タスクを完了させるために複数の関数呼び出しが独立して行われるシナリオに対応する。

まず、ツールをセットアップする。上記の映画の例とは異なり、これらの関数は呼び出されるために互いからの入力を必要としないので、並列呼び出しの良い候補となるはずである。


In [28]:
def power_disco_ball(power: bool) -> bool:
    """回転するディスコボールを動かす。"""
    print(f"ディスコボール {'スピンしています!' if power else '止まっています。'}")
    return True

def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """指定したパラメーターに合う音楽を再生する。

    Args:
      energetic: 音楽がエネルギッシュかどうか
      loud: 音楽がうるさいかうるさくないか
      bpm: 音楽の1分あたりの拍数

    Returns: 再生中の曲名。
    """
    print(f"音楽を再生 {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """照明を落として。

    Args:
      brightness: ライトの明るさ。0.0はオフ、1.0はフル。
    """
    print(f"{brightness:.0%}に設定されています")
    return True

house_fns = [power_disco_ball, start_music, dim_lights]

次に、指定されたすべての道具を使用できる命令でモデルを呼び出す。

In [29]:
# You generally set "mode": "any" to make sure Gemini actually *uses* the given tools.
party_chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": house_fns,
        "tool_config" : {
            "function_calling_config": {
                "mode": "any"
            }
        }
    }
)

# Call the API
response = party_chat.send_message(
    "Turn this place into a party!"
)


print_history(party_chat)

ディスコボール スピンしています!
50%に設定されています
音楽を再生 energetic=True loud=True, bpm=120
ディスコボール スピンしています!
50%に設定されています
音楽を再生 energetic=True loud=True, bpm=120
ディスコボール スピンしています!
50%に設定されています
音楽を再生 energetic=True loud=True, bpm=120
ディスコボール スピンしています!
50%に設定されています
音楽を再生 energetic=True loud=True, bpm=120
ディスコボール スピンしています!
50%に設定されています
音楽を再生 energetic=True loud=True, bpm=120
ディスコボール スピンしています!
50%に設定されています
音楽を再生 energetic=True loud=True, bpm=120
ディスコボール スピンしています!
50%に設定されています
音楽を再生 energetic=True loud=True, bpm=120
ディスコボール スピンしています!
50%に設定されています
音楽を再生 energetic=True loud=True, bpm=120
ディスコボール スピンしています!
50%に設定されています
音楽を再生 energetic=True loud=True, bpm=120
ディスコボール スピンしています!
50%に設定されています
音楽を再生 energetic=True loud=True, bpm=120
print_history


###user:

Turn this place into a party!

--------------------------------------------------------------------------------


###model:

...

Functional call: id=None args={'power': True} name='power_disco_ball'
Functional call: id=None args={'brightness': 0.5} name='dim_lights'
Functional call: id=None args={'bpm': 120, 'loud': True, 'energetic': True} name='start_music'
--------------------------------------------------------------------------------


###user:

Function response: will_continue=None scheduling=None parts=None id=None name='power_disco_ball' response={'result': True}
Function response: will_continue=None scheduling=None parts=None id=None name='dim_lights' response={'result': True}
Function response: will_continue=None scheduling=None parts=None id=None name='start_music' response={'result': 'Never gonna give you up.'}
--------------------------------------------------------------------------------


###model:

Functional call: id=None args={'power': True} name='power_disco_ball'
Functional call: id=None args={'brightness': 0.5} name='dim_lights'
Functional call: id=None args={'bpm': 120, 'loud': True, 'energetic': True} name='start_music'
--------------------------------------------------------------------------------


## 複合的なＦｕｎｃｔｉｏｎコール

このモデルは、複数のターンにわたって関数呼び出しを連鎖させることができ、1つの呼び出しの結果を次の呼び出しに反映させることができる。これにより、複雑な複数ステップの推論とタスクの完了が可能になります。

**例 特定の映画の上映時間を見つける**

theater_functionsを再利用し、より複雑なクエリを実行してみましょう。

In [30]:
chat = client.chats.create(
    model = MODEL_ID,
    config = {
        "tools": theater_functions,
    }
)

response = chat.send_message("""
  2025年01月01日にカリフォルニア州マウンテンビューで上映されるコメディ映画を検索。
  まず、映画のタイトルを見つけます。
  次に、それらの映画を上映している映画館を探します。
  最後に、各映画館の上映時間を検索します。
"""
)

print(response.text)
print("\n--- History ---")
print_history(chat)

2025年1月1日にカリフォルニア州マウンテンビューで上映されるコメディ映画の上映時間を検索しました。

**映画:**

*   **Barbie**
    *   Googleplex 16: 10:00, 11:00
    *   Android Theatre: 10:00, 11:00
*   **Oppenheimer** (注: この映画は通常コメディとは見なされませんが、検索結果に含まれていました。)
    *   Googleplex 16: 10:00, 11:00
    *   Android Theatre: 10:00, 11:00

--- History ---
print_history


###user:


  2025年01月01日にカリフォルニア州マウンテンビューで上映されるコメディ映画を検索。
  まず、映画のタイトルを見つけます。
  次に、それらの映画を上映している映画館を探します。
  最後に、各映画館の上映時間を検索します。


--------------------------------------------------------------------------------


###model:

...

###user:

Function response: will_continue=None scheduling=None parts=None id=None name='get_showtimes' response={'result': ['10:00', '11:00']}
Function response: will_continue=None scheduling=None parts=None id=None name='get_showtimes' response={'result': ['10:00', '11:00']}
Function response: will_continue=None scheduling=None parts=None id=None name='get_showtimes' response={'result': ['10:00', '11:00']}
Function response: will_continue=None scheduling=None parts=None id=None name='get_showtimes' response={'result': ['10:00', '11:00']}
--------------------------------------------------------------------------------


###model:

2025年1月1日にカリフォルニア州マウンテンビューで上映されるコメディ映画の上映時間を検索しました。

**映画:**

*   **Barbie**
    *   Googleplex 16: 10:00, 11:00
    *   Android Theatre: 10:00, 11:00
*   **Oppenheimer** (注: この映画は通常コメディとは見なされませんが、検索結果に含まれていました。)
    *   Googleplex 16: 10:00, 11:00
    *   Android Theatre: 10:00, 11:00

--------------------------------------------------------------------------------


ここでは、モデルがあなたの質問に答えるために7回の呼び出しを行い、その出力をその後の呼び出しと最終的な答えに使用していることがわかります。

## モードを使用したFunctionコール設定

AUTOモード（またはSDKのデフォルトの自動実行）で十分な場合が多いですが、モデル/チャットの初期化中またはsend_messageでtool_configパラメータを使用することで、モデルがいつどの関数を呼び出すかを正確に制御することができます。

**tool_config**は、FunctionCallingConfigを含むToolConfigオブジェクトを受け入れます。

**FunctionCallingConfig**には2つの主要なフィールドがあります：

* mode： モード：全体的な関数呼び出し動作を制御します（AUTO、ANY、NONE）。

* allowed_function_names： このターンでモデルが呼び出すことを制限される関数名のオプションのリスト。

## AUTO (デフォルトモード)
* Behavior： モデルはテキストで応答するか、提供されたツールから1つ以上の関数を呼び出すかを決定します。これは最も柔軟なモードです。

* SDKデフォルト： 自動実行を有効にしてChatSessionを使用する場合、tool_configで上書きされない限り、基本的な動作は実質的にAUTOモードを使用します。

In [31]:
chat = client.chats.create(model=MODEL_ID)

response = chat.send_message(
    message="Turn on the lights!",
    config={
        "system_instruction": instruction,
        "tools": light_controls,
        "tool_config" : types.ToolConfig(
            function_calling_config=types.FunctionCallingConfig(
                mode="auto"
            )
        )
    }
)

print_history(chat)

LIGHTBOT: Lights enabled.
print_history


###user:

Turn on the lights!

--------------------------------------------------------------------------------


###model:

...

###user:

Function response: will_continue=None scheduling=None parts=None id=None name='enable_lights' response={'result': None}
--------------------------------------------------------------------------------


###model:

承知いたしました。照明を点灯します。

--------------------------------------------------------------------------------


## NONE モード

Behavior： たとえツールが提供されていたとしても、モデルが関数を呼び出すことは明示的に禁止されている。テキストのみで応答する。純粋に会話のような反応が欲しいターンに便利。



In [32]:
none_chat = client.chats.create(model=MODEL_ID)

response = none_chat.send_message(
    message="Hello light-bot, what can you do?",
    config={
        "system_instruction": instruction,
        "tools": light_controls, # Tools are provided
        "tool_config" : types.ToolConfig(
            function_calling_config=types.FunctionCallingConfig(
                mode="none"
            )
        ) # but NONE mode prevents their use
    }
)

print_history(none_chat)

print_history


###user:

Hello light-bot, what can you do?

--------------------------------------------------------------------------------


###model:

こんにちは。私は照明を点灯したり消灯したりできます。そして、照明の色も設定できます。

--------------------------------------------------------------------------------


## ANYモード

* Behavior： モデルに少なくとも1つの関数を呼び出させます。

  * allowed_function_namesが設定されている場合、モデルはそのリストから1つ以上の関数を選択しなければなりません。

  * allowed_function_namesが設定されていない場合、モデルは完全なツールリストから1つ以上の関数を選択しなければなりません。

* 関数の自動呼び出しが有効な場合、SDKはmax_remote_calls（デフォルト：10）に達するまで関数を自動的に呼び出します。

* x回の自動関数呼び出しを許可するには、max_remote_callsをx + 1に設定します。もっと読む

* 使用例 アプリケーションの状態によって、次のステップで特定のアクションを実行する必要がある場合に便利です。

In [33]:
chat = client.chats.create(model=MODEL_ID)

response = chat.send_message(
    "Make this place PURPLE!",
    config={
        "system_instruction": instruction,
        "tools": light_controls, # Provide all tools
        "tool_config" : {
            "function_calling_config": {
                "mode": "any"
            }
        },
        "automatic_function_calling": {
            "maximum_remote_calls" : 1
        }
      } # But restrict to available_fns with ANY mode
)

print_history(chat)

LIGHTBOT: Lights set to #800080
print_history


###user:

Make this place PURPLE!

--------------------------------------------------------------------------------


###model:

Functional call: id=None args={'rgb_hex': '#800080'} name='set_light_color'
--------------------------------------------------------------------------------


###user:

Function response: will_continue=None scheduling=None parts=None id=None name='set_light_color' response={'result': None}
--------------------------------------------------------------------------------


###model:

Functional call: id=None args={'rgb_hex': '#800080'} name='set_light_color'
--------------------------------------------------------------------------------
